In [11]:
# Imports

import json
import geojson
import geopandas
import pandas as pd
import pathlib
import csv
import copy
import ast
import os
import time
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection

MAP_VERSION = 4
# STATS_TYPES = ['Count','Percent','nshft', 'pshft','growth', 'pshftna','growthna','Labels']
STATS_TYPES = ['Count','CountChange','Percent', 'PercentChange','SaturationChange', 'PercentChangeNatAdj','SaturationChangeNatAdj','Labels']

STATS_TYPE_LABELS = {
    'Count': "cnt",
    'CountChange': 'cch',
    'Percent': "pct",
    'PercentChange': "pch",
    'PercentChangeNatAdj': "pna",
    'SaturationChange': "sch",
    'SaturationChangeNatAdj': 'sna',
    'Labels': 'lbl'
}

# New to old mapping

# Count - count
# Percent - percent
# CountChange - nshft
# PercentChange - pshft
# SaturationChange - growth
# PercentChangeNatAdj - pshftna
# SaturationChangeNatAdj - growthna

In [12]:
ETHNICITIES = {
    "all": {
        "full": "All",
        "input_suffix": "All",
        "output_suffix": "al"
    },
    "asian": {
        "full": "Asian",
        "input_suffix": "Asi",
        "output_suffix": "as"
    },
    
    "black": {
        "full": "black",
        "input_suffix": "Bla",
        "output_suffix": "bl"
    },
    
    "hispanic": {
        "full": "Hispanic",
        "input_suffix": "His",
        "output_suffix": "hi"
    },
    
    "other": {
        "full": "Other",
        "input_suffix": "Oth",
        "output_suffix": "ot"
    },
    
    "white": {
        "full": "White",
        "input_suffix": "Whi",
        "output_suffix": "wh"
    },
    
    
}
ETHNICITY_KEYS = ETHNICITIES.keys()

def ethnicity_text(ethnicity,kind="input_suffix"):
    if kind == "input_suffix":
        return f'Eth{ETHNICITIES[ethnicity]["input_suffix"]}'
    elif kind == "output_suffix":
        return f'{ETHNICITIES[ethnicity]["output_suffix"]}'
    elif kind == "full":
        ETHNICITIES[ethnicity]["full"]
    else:
        ETHNICITIES[ethnicity]["full"]
        
print(ethnicity_text("all","output_suffix"))
print(ethnicity_text("white","input_suffix"))
print(ETHNICITY_KEYS)

al
EthWhi
dict_keys(['all', 'asian', 'black', 'hispanic', 'other', 'white'])


In [13]:
list(ETHNICITY_KEYS)[0]
STATS_TYPES[0]

'Count'

In [14]:
fips_codes = {
 '01': {'abbr': 'AL', 'full_name': 'alabama'},
 '02': {'abbr': 'AK', 'full_name': 'alaska'},
 '04': {'abbr': 'AZ', 'full_name': 'arizona'},
 '05': {'abbr': 'AR', 'full_name': 'arkansas'},
 '06': {'abbr': 'CA', 'full_name': 'california'},
 '08': {'abbr': 'CO', 'full_name': 'colorado'},
 '09': {'abbr': 'CT', 'full_name': 'connecticut'},
 '10': {'abbr': 'DE', 'full_name': 'delaware'},
 '11': {'abbr': 'DC', 'full_name': 'district_of_columbia'},
 '12': {'abbr': 'FL', 'full_name': 'florida'},
 '13': {'abbr': 'GA', 'full_name': 'georgia'},
 '15': {'abbr': 'HI', 'full_name': 'hawaii'},
 '16': {'abbr': 'ID', 'full_name': 'idaho'},
 '17': {'abbr': 'IL', 'full_name': 'illinois'},
 '18': {'abbr': 'IN', 'full_name': 'indiana'},
 '19': {'abbr': 'IA', 'full_name': 'iowa'},
 '20': {'abbr': 'KS', 'full_name': 'kansas'},
 '21': {'abbr': 'KY', 'full_name': 'kentucky'},
 '22': {'abbr': 'LA', 'full_name': 'louisiana'},
 '23': {'abbr': 'ME', 'full_name': 'maine'},
 '24': {'abbr': 'MD', 'full_name': 'maryland'},
 '25': {'abbr': 'MA', 'full_name': 'massachusetts'},
 '26': {'abbr': 'MI', 'full_name': 'michigan'},
 '27': {'abbr': 'MN', 'full_name': 'minnesota'},
 '28': {'abbr': 'MS', 'full_name': 'mississippi'},
 '29': {'abbr': 'MO', 'full_name': 'missouri'},
 '30': {'abbr': 'MT', 'full_name': 'montana'},
 '31': {'abbr': 'NE', 'full_name': 'nebraska'},
 '32': {'abbr': 'NV', 'full_name': 'nevada'},
 '33': {'abbr': 'NH', 'full_name': 'new_hampshire'},
 '34': {'abbr': 'NJ', 'full_name': 'new_jersey'},
 '35': {'abbr': 'NM', 'full_name': 'new_mexico'},
 '36': {'abbr': 'NY', 'full_name': 'new_york'},
 '37': {'abbr': 'NC', 'full_name': 'north_carolina'},
 '38': {'abbr': 'ND', 'full_name': 'north_dakota'},
 '39': {'abbr': 'OH', 'full_name': 'ohio'},
 '40': {'abbr': 'OK', 'full_name': 'oklahoma'},
 '41': {'abbr': 'OR', 'full_name': 'oregon'},
 '42': {'abbr': 'PA', 'full_name': 'pennsylvania'},
 '44': {'abbr': 'RI', 'full_name': 'rhode_island'},
 '45': {'abbr': 'SC', 'full_name': 'south_carolina'},
 '46': {'abbr': 'SD', 'full_name': 'south_dakota'},
 '47': {'abbr': 'TN', 'full_name': 'tennessee'},
 '48': {'abbr': 'TX', 'full_name': 'texas'},
 '49': {'abbr': 'UT', 'full_name': 'utah'},
 '50': {'abbr': 'VT', 'full_name': 'vermont'},
 '51': {'abbr': 'VA', 'full_name': 'virginia'},
 '53': {'abbr': 'WA', 'full_name': 'washington'},
 '54': {'abbr': 'WV', 'full_name': 'west_virginia'},
 '55': {'abbr': 'WI', 'full_name': 'wisconsin'},
 '56': {'abbr': 'WY', 'full_name': 'wyoming'}}

In [15]:
STATE_FILES = [
    "tl_2010_01_bg10.json",
    "tl_2010_02_bg10.json",
    "tl_2010_04_bg10.json",
    "tl_2010_05_bg10.json",
    "tl_2010_06_bg10.json",
    "tl_2010_08_bg10.json",
    "tl_2010_09_bg10.json",
    "tl_2010_10_bg10.json",
    "tl_2010_11_bg10.json",
    "tl_2010_12_bg10.json",
    "tl_2010_13_bg10.json",
    "tl_2010_15_bg10.json",
    "tl_2010_16_bg10.json",
    "tl_2010_17_bg10.json",
    "tl_2010_18_bg10.json",
    "tl_2010_19_bg10.json",
    "tl_2010_20_bg10.json",
    "tl_2010_21_bg10.json",
    "tl_2010_22_bg10.json",
    "tl_2010_23_bg10.json",
    "tl_2010_24_bg10.json",
    "tl_2010_25_bg10.json",
    "tl_2010_26_bg10.json",
    "tl_2010_27_bg10.json",
    "tl_2010_28_bg10.json",
    "tl_2010_29_bg10.json",
    "tl_2010_30_bg10.json",
    "tl_2010_31_bg10.json",
    "tl_2010_32_bg10.json",
    "tl_2010_33_bg10.json",
    "tl_2010_34_bg10.json",
    "tl_2010_35_bg10.json",
    "tl_2010_36_bg10.json",
    "tl_2010_37_bg10.json",
    "tl_2010_38_bg10.json",
    "tl_2010_39_bg10.json",
    "tl_2010_40_bg10.json",
    "tl_2010_41_bg10.json",
    "tl_2010_42_bg10.json",
    "tl_2010_44_bg10.json",
    "tl_2010_45_bg10.json",
    "tl_2010_46_bg10.json",
    "tl_2010_47_bg10.json",
    "tl_2010_48_bg10.json",
    "tl_2010_49_bg10.json",
    "tl_2010_50_bg10.json",
    "tl_2010_51_bg10.json",
    "tl_2010_53_bg10.json",
    "tl_2010_54_bg10.json",
    "tl_2010_55_bg10.json",
    "tl_2010_56_bg10.json",
]

In [16]:
# Input Files
# 2 => "02"
def formatted_state_fips_code(fips_number):
    return f'{fips_number:02d}'

#  block_group_geojson_filename(state_fips_code=46, year=2010)
# tl_2010_46_bg10.json
def block_group_geojson_filename(state_fips_code=1, census_year=2010):
    year_string = str(census_year)
    fips_code_string = formatted_state_fips_code(state_fips_code)
    return f'tl_{year_string}_{fips_code_string}_bg10.json'

# wa_washington_zip_codes_geo.min.json
# def zip_code_geojson_filename(state_fips_code=1):
#     formatted_fips_code = formatted_state_fips_code(state_fips_code)
#     state_abbr = fips_codes[formatted_fips_code]["abbr"].lower()
#     state_full_name = fips_codes[formatted_fips_code]["full_name"]
#     return f'{state_abbr}_{state_full_name}_zip_codes_geo.min.json'


# def zip_geojson_files_for_market(market_data):
#     fips_codes = json.loads(market_data['StateFips'])
#     files = []
#     for fips_code in fips_codes:
#         files.append(zip_code_geojson_filename(fips_code))
#     return files

def bg_geojson_files_for_market(market_data, census_year):
    fips_codes = json.loads(market_data['StateFips'])
    files = []
    for fips_code in fips_codes:
        files.append(block_group_geojson_filename(state_fips_code=int(fips_code), census_year=census_year))
    return files

def read_geo_gson_file(file_path):
    with open(file_path) as f:
        gj = geojson.load(f)
        features = gj['features']
    print(f'Loaded {len(features)} features from {file_path}')
    return  features

def read_geojson_files(file_paths, base_path = "./"):
    features = []
    for file_path in file_paths:
        features += read_geo_gson_file(os.path.join(base_path, file_path))
    return features
    

# def read_market_zip_code_usage(market_file_prefix, base_path="./source_data" ):
#     data_type = "zip"
#     csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
#     data_rows = []
#     with open(csv_file_path, encoding='utf-8-sig') as csvfile:
#         reader = csv.DictReader(csvfile)
#         for row in reader:
#             data_rows.append(row)
#     zip_codes = []
#     for data_row in data_rows:
#         zip_codes.append(data_row['Zip'].zfill(5))
#     print(f'Loaded {len(zip_codes)} zip_codes from {csv_file_path}')
#     return zip_codes
        
def read_market_block_group_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "bg"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    block_groups = []
    for data_row in data_rows:
        block_groups.append(data_row['BG'].zfill(12))
    print(f'Loaded {len(block_groups)} block groups from {csv_file_path}')
    return block_groups
        
# market_file_prefix = 'DallasFortWorth'
# market_prefix= 'dfw'
# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes

def file_type_for_data_type(data_type = "bg"):
    if data_type == "zip": 
        return "Zip"
    if data_type == "bg":
        return "BG"

# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
def mapping_data_file_path_old(market_file_prefix,base_path="./source_data", data_type = "bg"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}_Mapping_{file_type}_Data.csv'

def mapping_data_file_path(market_file_prefix, ethnicity, stats_type = "Count", base_path="./source_data"):
    ethnicity_suffix = ethnicity_file_suffix = ethnicity_text(ethnicity,"input_suffix")
    return f'{base_path}/{market_file_prefix}_{stats_type}_{ethnicity_suffix}.csv'

# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes
def map_settings_data_file_path_old(market_file_prefix,ethnicity, base_path="./source_data", stats_type = "Count"):
    ethnicity_suffix = ethnicity_file_suffix = ethnicity_text(ethnicity,"input_suffix")
    return f'{base_path}/{market_file_prefix}_Mapping_Settings_Transposed_File_{stats_type}_{ethnicity_suffix}.csv'

def map_settings_data_file_path(market_file_prefix,ethnicity,  stats_type = "Count",base_path="./source_data",):
    ethnicity_suffix = ethnicity_file_suffix = ethnicity_text(ethnicity,"input_suffix")
    return f'{base_path}/{market_file_prefix}_Mapping_Settings_Transposed_File_{stats_type}_{ethnicity_suffix}.csv'


def published_map_data_path_old(market_prefix, ethnicity, data_type = "bg", published_data_base_path="./public"):
    ethnicity_suffix = ethnicity_text(ethnicity,"output_suffix")
    return f'{published_data_base_path}/{market_prefix}/data/{market_prefix}_{ethnicity_suffix}_{data_type}_data_v{MAP_VERSION}.json'

def published_map_data_path(market_prefix, ethnicity, stats_type = "Count", published_data_base_path="./public"):
    ethnicity_suffix = ethnicity_text(ethnicity,"output_suffix")
    stats_type_suffix = STATS_TYPE_LABELS[stats_type]
    return f'{published_data_base_path}/{market_prefix}/data/{market_prefix}_{ethnicity_suffix}_{stats_type_suffix}_data_v{MAP_VERSION}.json'


def published_map_settings_path(market_prefix, ethnicity, stats_type = "Count", published_data_base_path="./public"):
    ethnicity_suffix = ethnicity_text(ethnicity,"output_suffix")
    stats_type_suffix = STATS_TYPE_LABELS[stats_type]
    return f'{published_data_base_path}/{market_prefix}_{ethnicity_suffix}_{stats_type_suffix}_settings_data_v{MAP_VERSION}.json'

def published_map_labels_path_old(market_prefix, ethnicity, data_type = "bg", published_data_base_path="./public"):
    ethnicity_suffix = ethnicity_text(ethnicity,"output_suffix")
    return f'{published_data_base_path}/{market_prefix}_{ethnicity_suffix}_labels_v{MAP_VERSION}.json'

def published_map_labels_path(market_prefix, published_data_base_path="./public"):

    return f'{published_data_base_path}/{market_prefix}_labels_v{MAP_VERSION}.json'

def read_mapping_data(market_file_prefix,ethnicity, stats_type='Count', base_path='./source_data'):
    data_rows = []    
    csv_file_path = mapping_data_file_path(market_file_prefix, ethnicity, stats_type = stats_type, base_path=base_path)
    with open(csv_file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')
    return data_rows

def create_zip_features(features, zip_data_rows):
    new_features_to_keep = []
    for data_row in zip_data_rows:
        zip_code = data_row['Zip'].zfill(5)
        # find the associated feature
        for feature in features:
            if feature['properties']["ZCTA5CE10"] == zip_code:
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'Zip':
                        if data_row[key]:
                            if data_row[key] != 'inf':
                                new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_bg_features(features, map_settings_data, bg_data_rows, stats_type):
    new_features_to_keep = []
    for data_row in bg_data_rows:
        block_group = str(data_row['BG']).zfill(12)
        # find the associated feature
        found = False
        if block_group in features:
            found = True
            feature = features[block_group]
            new_feature = copy.deepcopy(feature)
            properties = new_feature['properties']
            properties.pop("ALAND10",0)
            properties.pop("AWATER10",0)
            properties.pop("BLKGRPCE10",0)
            properties.pop("COUNTYFP10",0)
            properties.pop("FUNCSTAT10",0)
            properties.pop("MTFCC10",0)
            properties.pop("NAMELSAD10",0)
            properties.pop("STATEFP10",0)
            properties.pop("TRACTCE10",0)
            new_feature['properties'] = properties
            for key in list(data_row.keys())[0:]:
                if key not in  ['BG','LAT','LONG']: #and 'Count' in key:
                    if data_row[key] and data_row[key] != 'inf':
                        translated_key = map_settings_data[key]['mapbox_segment']
                        value = ast.literal_eval(data_row[key].replace(',',''))
                        value = int(value * 1000000) / 1000000
                        new_feature['properties'][translated_key] =  value
            new_features_to_keep.append(new_feature)
    #         if found == False:
    #             print(f'failed: {block_group}')
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_geojson(features):
    print(f'Created GeoJson with  {len(features)} features')
    return FeatureCollection(features)


def create_data_json(features):
    data = []
    for feature in features:
        #centroid = {
        #    "GEOID10": feature["properties"]["GEOID10"],
        #    "INTPTLAT10": feature["properties"]["INTPTLAT10"],
        #    "INTPTLON10": feature["properties"]["INTPTLON10"],
        #}
        datum = feature["properties"]
        data.append(datum)
    return data

def write_geojson(geojson_data, market_prefix,ethnicity, stats_type='Count',published_data_base_path = "./public"):
    geojson_string = geojson.dumps(geojson_data, sort_keys=True)
    ethnicity_suffix = ethnicity_file_suffix = ethnicity_text(ethnicity,"output_suffix")
    f = open(f'{published_data_base_path}/{market_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_v{MAP_VERSION}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geojson data to  {published_data_base_path}/{market_prefix}/data/{market_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_v{MAP_VERSION}.json')
    
def write_data_json(json_data, market_prefix, ethnicity, stats_type='Count',published_data_base_path = "./public"):
    json_string = json.dumps(json_data, sort_keys=True)
    ethnicity_suffix = ethnicity_file_suffix = ethnicity_text(ethnicity,"output_suffix")
    f = open(f'{published_data_base_path}/{market_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_data_v{MAP_VERSION}.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote centroid json data to  {published_data_base_path}/{market_prefix}/data/{market_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_data_v{MAP_VERSION}.json')

# TODO
def read_map_settings(market_file_prefix, ethnicity, stats_type = "Count", base_path="./source_data"):
    data_rows = []
    csv_file_path = map_settings_data_file_path(market_file_prefix,ethnicity, stats_type = stats_type, base_path=base_path )
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')   
    return data_rows

def write_settings_data(settings_data, market_file_prefix, ethnicity, stats_type = "Count", base_path="./public"):
    processed_settings_data = OrderedDict()
    segment_number = 0
    for row in settings_data:
        segment_number += 1
        del row["SeriesLongUnique"]
        if row["Display_Data_Type"] == "HH_Count":
            row["Display_Data_Type"] = "HH Count"
        mapbox_segment_key = f's{segment_number}'
        row['mapbox_segment'] = mapbox_segment_key
        processed_settings_data[row['SeriesShortUnique']] = row
    json_string = json.dumps(processed_settings_data, indent = 2, sort_keys=False)

    settings_data_file_path = published_map_settings_path(market_file_prefix, ethnicity, stats_type = stats_type, published_data_base_path=base_path)
    f = open(settings_data_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote settings data rows to {settings_data_file_path}') 
    return processed_settings_data
    

def create_bg_labels(market_file_prefix, base_path="./source_data" ,  published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    ethnicity = list(ETHNICITY_KEYS)[0]
    stats_type = STATS_TYPES[0]
    csv_file_path = mapping_data_file_path(market_file_prefix, ethnicity, stats_type, base_path=base_path)
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['BG']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['LONG']), float(data_row['LAT']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_file_prefix, published_data_base_path=published_data_base_path)
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_geojson_settings(csv_file_path, published_data_base_path = "./public"):

    geospatial_settings = OrderedDict()
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings_v{MAP_VERSION}.csv'
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            row['CenterLat'] = float(row['CenterLat'])
            row['CenterLong'] = float(row['CenterLong'])
            geospatial_settings[row['MarketName']] = row
    geojson_string = json.dumps(geospatial_settings, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/geospatial_settings_v{MAP_VERSION}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geospatials settings data to  {published_data_base_path}/geospatial_settings_v{MAP_VERSION}.json')
    
    
def create_tileset_data_recipe_file(market_file_prefix, ethnicity, stats_type, published_data_base_path):
    ethnicity_suffix = ethnicity_file_suffix = ethnicity_text(ethnicity,"output_suffix")

    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
          'tiles': {
            'layer_size': 2500
          }
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
    

def create_tileset_labels_recipe_file(market_file_prefix, published_data_base_path):


    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{market_file_prefix}_labels_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{market_file_prefix}_labels_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
            
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{market_file_prefix}_labels_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
        
# def mstds_convert_file(market_file_prefix, published_data_base_path, filetype='data'):
#     recipe_file_path = os.path.join(published_data_base_path, market_file_prefix,'labels','mts-recipe.json' )
#     market_file_prefix, published_data_base_path
    

In [17]:
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
def create_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    nationwide_features = {}
    for file in STATE_FILES:
        print(f'processing: {file}')
        geosjon_path = os.path.join(geojson_files_base_path,file)
        features = read_geo_gson_file(geosjon_path)
        for feature in features:
            key = feature['properties']["GEOID10"]
            nationwide_features[key] = feature
    json_string = json.dumps(nationwide_features, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson data to  {geojson_files_base_path}/nationwide_bg.json')

def read_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"nationwide_bg.json")
    # Opening JSON file
    f = open(geosjon_path)
    # returns JSON object as 
    # a dictionary
    features = json.load(f)
    # Closing file
    f.close()
    return features

def write_nation_bg_centroids_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"nationwide_bg.json")
    features = read_nation_bg_features_json(geojson_files_base_path)
    centroids = []
    for key in features.keys:
        feature = features[key]
        centroid = {
            "GEOID10": key,
            "INTPTLAT10": feature["properties"]["INTPTLAT10"],
            "INTPTLON10": feature["properties"]["INTPTLON10"],
        }
        centroids.append(centroid)
    json_string = json.dumps(centroids, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg_centroids.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote bg centroids data to  {geojson_files_base_path}/nationwide_bg_centroids.json')
    
        
        
    

In [18]:
# ETHNICITY TODO
def create_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        # segment = row['Segment']
        year = row['Year']
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(region,year)
        print(urban, suburban, exurban)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        if region not in data_json.keys():
            data_json[region] = {'displayRegion': display_region, 'data': {}}
        if year not in data_json[region]['data'].keys():
            data_json[region]['data'][year] = {}
        if 'Urban' not in data_json[region]['data'][year].keys():
            data_json[region]['data'][year]['Urban'] = {}
            data_json[region]['data'][year]['SubUrban'] = {}
            data_json[region]['data'][year]['ExUrban'] = {}
        data_json[region]['data'][year]['Urban'] = urban
        data_json[region]['data'][year]['SubUrban'] = suburban
        data_json[region]['data'][year]['ExUrban'] = exurban
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json')



In [19]:
# ETHNICITY TODO
def create_comparison_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        display_region = row['DisplayRegion']
        year = row['Year']
        age_segment=row['Display_Age_Segment']
        income_segment=row['Display_Income_Segment']
        key = f'{region}-{year}-{age_segment}-{income_segment}'
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(key)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        isMetro = row['IsMetro']
        data_json[key] = {
            "year": year,
            "ageSegment": age_segment,
            "incomeSegment": income_segment,
            "urban": urban,
            "suburban": suburban,
            "exurban": exurban,
            "isMetro": isMetro,
        }
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json')
    

In [20]:
# ETHNICITY TODO
def create_data_options_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    year_options_set = set()
    age_segment_options_set = set()
    income_segment_options_set = set()
    region_options_list = []
    for row in chart_data_rows:
        year_options_set.add(row["Year"])
        age_segment_options_set.add(row["Display_Age_Segment"])
        income_segment_options_set.add(row["Display_Income_Segment"])
        region = {
            "region": row["Region"],
            "displayRegion": row["DisplayRegion"],
            "isMetro": row["IsMetro"],
        }
        region_options_list.append(region)
    year_options_list = list(year_options_set)
    year_options_list.sort()
    age_segment_options_list = list(age_segment_options_set)
    age_segment_options_list.sort()
    income_segment_options_list = list(income_segment_options_set)
    income_segment_options_list.sort()
    region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')
    dataOptions = {
        "yearOptions": year_options_list,
        'ageSegmentOptions': age_segment_options_list,
        'incomeSegmentOptions': income_segment_options_list,
        "regionOptions": region_options
    }
    data_options_string = json.dumps(dataOptions, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/data_options_v{MAP_VERSION}.json', "w")
    f.write(data_options_string)
    f.close()
    print(f'Wrote data options data to  {published_data_base_path}/data_options_v{MAP_VERSION}.json')


In [21]:
def process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year):
    nationwide_bg_features = read_nation_bg_features_json(geojson_files_base_path)
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings_v{MAP_VERSION}.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows:
        #         if market_data['MarketName'] in ['Texas','DallasFortWorthTX']:
        #             #if True:
        #             #print(market_data['MarketName'])
        #if market_data['MarketName'] in ['Ohio','ClevelandOH']:
        # if market_data['MarketName'] in ['HomosassaSprngFL']:
        # if True:
        #if market_data['MarketName'] in ['AbileneTX']:
        if True:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MarketName'].lower()
            market_prefix = market_file_prefix
            mapbox_market_prefix = market_data['MapboxMarketName'].lower()
            create_tileset_labels_recipe_file(market_prefix, published_data_base_path)
            create_bg_labels(market_file_prefix, base_path=source_data_base_path , published_data_base_path=published_data_base_path)
            for ethnicity in ETHNICITIES.keys():
                for stats_type in STATS_TYPES[:-1]:
                    create_tileset_data_recipe_file(market_prefix, ethnicity, stats_type, published_data_base_path)
                    map_settings_data = read_map_settings(market_data['MarketName'], ethnicity, stats_type , base_path=source_data_base_path)
                    map_settings = write_settings_data(map_settings_data, market_prefix, ethnicity, stats_type, base_path=published_data_base_path)
                    block_group_rows = read_mapping_data(market_file_prefix, ethnicity,stats_type, source_data_base_path)
                    features_to_use = create_bg_features(nationwide_bg_features, map_settings, block_group_rows, stats_type)
                    new_geojson = create_geojson(features_to_use)
                    write_geojson(new_geojson, market_prefix, ethnicity, stats_type,published_data_base_path=published_data_base_path)
                    if stats_type.lower() != 'labels':
                        new_data_json = create_data_json(features_to_use)
                        write_data_json(new_data_json, market_prefix.lower(), ethnicity, stats_type, published_data_base_path)
            market_count += 1
    # Finished states - create overall settings file    
    create_geojson_settings(csv_file_path, published_data_base_path = "./public")
    print(f'*** COMPLETED {market_count} markets')
    print(completed_markets)
                          
        
# source_data_base_path = "./source_data"
# source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie/"
# published_data_base_path = "./public"
# geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# census_year = 2010

In [22]:
STATS_TYPES[:2]

['Count', 'CountChange']

In [24]:
# Set up input and output locations
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# Source csv data path
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie"
# Output data path - all of the files in this folder should be uploaded to the cms GeoSpatialTool/v1 folder
published_data_base_path = "./public"
published_data_base_path = f'/Users/alanmccann/Dropbox/bain/mtsds-data/v{MAP_VERSION}'
# Census year for geojson
census_year = 2010


# Process the data
process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year)
# create_chart_json(published_data_base_path)
# create_comparison_chart_json(published_data_base_path)
# create_data_options_json(published_data_base_path)


Market Data: 1
Wrote geojson labels data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama_labels_v4.json
Loaded 64 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Alabama_Mapping_Settings_Transposed_File_Count_EthAll.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama_al_cnt_settings_data_v4.json
Loaded 3438 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/alabama_Count_EthAll.csv
Processed 3438 features to use
Created GeoJson with  3438 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_al_cnt_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_al_cnt_data_v4.json
Loaded 48 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Alabama_Mapping_Settings_Transposed_File_CountChange_EthAll.csv
Wrote settings data 

Processed 3438 features to use
Created GeoJson with  3438 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_as_sch_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_as_sch_data_v4.json
Loaded 48 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Alabama_Mapping_Settings_Transposed_File_PercentChangeNatAdj_EthAsi.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama_as_pna_settings_data_v4.json
Loaded 3438 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/alabama_PercentChangeNatAdj_EthAsi.csv
Processed 3438 features to use
Created GeoJson with  3438 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_as_pna_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_as_pna_data

Processed 3438 features to use
Created GeoJson with  3438 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_hi_pct_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_hi_pct_data_v4.json
Loaded 48 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Alabama_Mapping_Settings_Transposed_File_PercentChange_EthHis.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama_hi_pch_settings_data_v4.json
Loaded 3438 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/alabama_PercentChange_EthHis.csv
Processed 3438 features to use
Created GeoJson with  3438 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_hi_pch_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_hi_pch_data_v4.json
Loa

Processed 3438 features to use
Created GeoJson with  3438 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_wh_cnt_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_wh_cnt_data_v4.json
Loaded 48 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Alabama_Mapping_Settings_Transposed_File_CountChange_EthWhi.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama_wh_cch_settings_data_v4.json
Loaded 3438 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/alabama_CountChange_EthWhi.csv
Processed 3438 features to use
Created GeoJson with  3438 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_wh_cch_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alabama/data/alabama_wh_cch_data_v4.json
Loaded 

Processed 534 features to use
Created GeoJson with  534 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alaska/data/alaska_al_pna_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alaska/data/alaska_al_pna_data_v4.json
Loaded 48 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Alaska_Mapping_Settings_Transposed_File_SaturationChangeNatAdj_EthAll.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alaska_al_sna_settings_data_v4.json
Loaded 534 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/alaska_SaturationChangeNatAdj_EthAll.csv
Processed 534 features to use
Created GeoJson with  534 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alaska/data/alaska_al_sna_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/alaska/data/alaska_al_sna_data_v4.json
L

...

Processed 4489 features to use
Created GeoJson with  4489 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsin/data/wisconsin_ot_cnt_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsin/data/wisconsin_ot_cnt_data_v4.json
Loaded 48 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Wisconsin_Mapping_Settings_Transposed_File_CountChange_EthOth.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsin_ot_cch_settings_data_v4.json
Loaded 4489 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/wisconsin_CountChange_EthOth.csv
Processed 4489 features to use
Created GeoJson with  4489 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsin/data/wisconsin_ot_cch_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsin/data/wisconsin_ot_cc

Processed 4489 features to use
Created GeoJson with  4489 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsin/data/wisconsin_wh_pna_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsin/data/wisconsin_wh_pna_data_v4.json
Loaded 48 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Wisconsin_Mapping_Settings_Transposed_File_SaturationChangeNatAdj_EthWhi.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsin_wh_sna_settings_data_v4.json
Loaded 4489 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/wisconsin_SaturationChangeNatAdj_EthWhi.csv
Processed 4489 features to use
Created GeoJson with  4489 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsin/data/wisconsin_wh_sna_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wisconsi

Processed 410 features to use
Created GeoJson with  410 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_as_pch_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_as_pch_data_v4.json
Loaded 48 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Wyoming_Mapping_Settings_Transposed_File_SaturationChange_EthAsi.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming_as_sch_settings_data_v4.json
Loaded 410 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/wyoming_SaturationChange_EthAsi.csv
Processed 410 features to use
Created GeoJson with  410 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_as_sch_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_as_sch_data_v4.json
Lo

Processed 410 features to use
Created GeoJson with  410 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_hi_cch_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_hi_cch_data_v4.json
Loaded 64 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Wyoming_Mapping_Settings_Transposed_File_Percent_EthHis.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming_hi_pct_settings_data_v4.json
Loaded 410 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/wyoming_Percent_EthHis.csv
Processed 410 features to use
Created GeoJson with  410 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_hi_pct_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_hi_pct_data_v4.json
Loaded 48 data rows 

Processed 410 features to use
Created GeoJson with  410 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_ot_sna_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_ot_sna_data_v4.json
Loaded 64 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/Wyoming_Mapping_Settings_Transposed_File_Count_EthWhi.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming_wh_cnt_settings_data_v4.json
Loaded 410 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan10_Run2 Brian Stobie/wyoming_Count_EthWhi.csv
Processed 410 features to use
Created GeoJson with  410 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_wh_cnt_v4.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/v4/wyoming/data/wyoming_wh_cnt_data_v4.json
Loaded 48 data rows from

In [ ]:
def upload_to_mapbox(source_data_base_path,published_data_base_path):
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings_v{MAP_VERSION}.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows: #[0:10]:
        # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
        if True:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MapboxMarketName'].lower()
            os.system(f'cd {published_data_base_path}; tilesets upload-source stobieb {market_file_prefix}_bg_data-src_v{MAP_VERSION} {market_file_prefix}_bg_data_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION} --recipe {market_file_prefix}-bg-recipe_v{MAP_VERSION}.json --name "{market_file_prefix}_bg_data"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION}')
            os.system(f'cd {published_data_base_path};  tilesets upload-source stobieb {market_file_prefix}_bg_labels-src {market_file_prefix}_bg_labels_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_labels --recipe {market_file_prefix}-labels-recipe.json --name "{market_file_prefix}_bg_labels_v{MAP_VERSION}"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_labels')

In [14]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}

import subprocess
# tilesets job stobieb.dallasfortworth_bg_data cl9kgdofo000209l52zyy3mdi
process = subprocess.Popen(['tilesets', 'job', 'stobieb.dallasfortworth_bg_data', 'cl9kgdofo000209l52zyy3mdi'],
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)
stdout, stderr = process.communicate()
output = stdout.decode('utf-8').split('\n')
error = stderr.decode('utf-8')
result = json.loads(output[0])



In [15]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}
published_data_base_path = f'/Users/alanmccann/Dropbox/bain/mtsds-data/v{MAP_VERSION}'
import subprocess

cwd = os.getcwd()
os.chdir(published_data_base_path)

market_file_prefix = 'austin'
stats_type = 'Count'

def upload_source(market_file_prefix, ethnicity, stats_type, working_directory):
    ethnicity_suffix = ethnicity_text(ethnicity, "output_suffix")
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_src_v{MAP_VERSION}',
        f'{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_v{MAP_VERSION}.json',
        '--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)

def upload_label_source(market_file_prefix, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_labels_src_v{MAP_VERSION}',
        f'{market_file_prefix}_labels_v{MAP_VERSION}.json',
        #'--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)
    
# upload_source(market_file_prefix, stats_type, published_data_base_path)

In [16]:
def update_recipe(market_file_prefix, ethnicity, stats_type, working_directory):
    ethnicity_suffix = ethnicity_text(ethnicity, "output_suffix")
    cwd = os.getcwd()
    os.chdir(working_directory)
    update_recipe_command = [
        'tilesets',
        'update-recipe',
        f'stobieb.{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_v{MAP_VERSION}',
        f'{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_recipe_v{MAP_VERSION}.json',
        
    ]

    print(' '.join(update_recipe_command))
    process = subprocess.Popen(update_recipe_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    
def update_label_recipe(market_file_prefix,  working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    update_recipe_command = [
        'tilesets',
        'update-recipe',
        f'stobieb.{market_file_prefix}_labels_v{MAP_VERSION}',
        f'{market_file_prefix}_labels_recipe_v{MAP_VERSION}.json',
        
    ]

    print(' '.join(update_recipe_command))
    process = subprocess.Popen(update_recipe_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    
def upload_tileset(market_file_prefix, ethnicity, stats_type, working_directory):
    ethnicity_suffix = ethnicity_text(ethnicity, "output_suffix")
    cwd = os.getcwd()
    os.chdir(working_directory)
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_v{MAP_VERSION}',
        '--recipe', 
        f'{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_v{MAP_VERSION}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

def upload_label_tileset(market_file_prefix, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_labels_v{MAP_VERSION}',
        '--recipe', f'{market_file_prefix}_labels_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix}_labels_v{MAP_VERSION}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

In [17]:
import time
def publish_tileset(market_file_prefix, ethnicity, stats_type, working_directory):
    ethnicity_suffix = ethnicity_text(ethnicity, "output_suffix")
    cwd = os.getcwd()
    os.chdir(working_directory)
    publish_tileset_command = [
        'tilesets',
        'publish',
        f'stobieb.{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_v{MAP_VERSION}'
    ]

    print(' '.join(publish_tileset_command))
    print(datetime.datetime.now())
    process = subprocess.Popen(publish_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    result = json.loads(output[0])
    print(result)
    get_job_status_command = [
        'tilesets',
        'job',
        f'stobieb.{market_file_prefix}_{ethnicity_suffix}_{STATS_TYPE_LABELS[stats_type]}_v{MAP_VERSION}',
        result['jobId']
    ]
    state = 'processing'
    while state == 'processing':
        process = subprocess.Popen(get_job_status_command,
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)

        stdout, stderr = process.communicate()
        output = stdout.decode('utf-8').split('\n')
        try:
            result = json.loads(output[0])
        except:
            result = {}
        if 'stage' in result.keys():
            state = result['stage']
        if state != 'success':
            print(f'{state}: sleeping')
            time.sleep(15)
    print('finished')
    print(datetime.datetime.now())

def publish_label_tileset(market_file_prefix,  working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    publish_tileset_command = [
        'tilesets',
        'publish',
        f'stobieb.{market_file_prefix}_labels_v{MAP_VERSION}'
    ]

    print(' '.join(publish_tileset_command))
    print(datetime.datetime.now())
    process = subprocess.Popen(publish_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    result = json.loads(output[0])
    print(result)
    get_job_status_command = [
        'tilesets',
        'job',
        f'stobieb.{market_file_prefix}_labels_v{MAP_VERSION}',
        result['jobId']
    ]
    state = 'processing'
    while state == 'processing':
        process = subprocess.Popen(get_job_status_command,
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)

        stdout, stderr = process.communicate()
        output = stdout.decode('utf-8').split('\n')
        try:
            result = json.loads(output[0])
        except:
            result = {}
        if 'stage' in result.keys():
            state = result['stage']
        if state != 'success':
            print(f'{state}: sleeping')
            time.sleep(15)
    print('finished')
    print(datetime.datetime.now())

In [18]:
import csv
import datetime
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_Jan7_Run2 Brian Stobie"
published_data_base_path = f'/Users/alanmccann/Dropbox/bain/mtsds-data/v{MAP_VERSION}'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings_v{MAP_VERSION}.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)
market_count = 0
completed_markets = []
# STATS_TYPES = ['count','percent','nshft','labels','pshft','pshftna','growthna']
# STATS_TYPES = ['growth']
# STATS_TYPES = ['Count','Percent']
# MARKETS = ["ColoradoSpringCO","DallasFortWrthTX","EastStroudsbrgPA","HiltonHeadIlndSC","HomosassaSprngFL","DstrctOfColumbia"]
# MARKETS = ["ClevelandOH"]

# STATS_TYPES = ['pshftna','growthna']
# STATS_TYPES = ['nshft']
# Ohio market_rows[416:417]
# Cleveland market_rows[73:74]
count = 77
for market_data in market_rows:  #[416:417]:
    if market_data['MarketName'] in ['Virginia']:
        # Create labels
        market_file_prefix = market_data['MarketName'].lower()
        upload_label_source(market_file_prefix, published_data_base_path)
        upload_label_tileset(market_file_prefix, published_data_base_path)
        update_label_recipe(market_file_prefix, published_data_base_path)
        publish_label_tileset(market_file_prefix,published_data_base_path)
        # Create data
        for ethnicity in list(ETHNICITIES.keys()):
            if True: #market_data['MarketName'] in MARKETS: #,'DallasFortWorthTX']:
                count += 1
                print('count',count)
                print('creating tilesets for:', market_file_prefix, ethnicity)
                print(datetime.datetime.now())
                for stats_type in STATS_TYPES[:-1]:
                    upload_source(market_file_prefix, ethnicity, stats_type, published_data_base_path)
                    upload_tileset(market_file_prefix, ethnicity,  stats_type, published_data_base_path)
                    update_recipe(market_file_prefix,ethnicity,  stats_type, published_data_base_path)
                    publish_tileset(market_file_prefix,ethnicity,  stats_type, published_data_base_path)

        
        

tilesets upload-source stobieb virginia_labels_src_v4 virginia_labels_v4.json
['upload progress', '{"id": "mapbox://tileset-source/stobieb/virginia_labels_src_v4", "files": 2, "source_size": 1286578, "file_size": 643289}', '']
tilesets create stobieb.virginia_labels_v4 --recipe virginia_labels_recipe_v4.json --name virginia_labels_v4
['{"message": "stobieb.virginia_labels_v4 already exists"}', ''] 
tilesets update-recipe stobieb.virginia_labels_v4 virginia_labels_recipe_v4.json
[''] Updated recipe.

tilesets publish stobieb.virginia_labels_v4
2024-01-09 15:39:27.835091
['{"message": "Processing stobieb.virginia_labels_v4", "jobId": "clr6tewjh000e08l45f5h9xmh"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.virginia_labels_v4 or run tilesets job stobieb.virginia_labels_v4 clr6tewjh000e08l45f5h9xmh to view the status of your tileset.

{'message': 'Processing stobieb.virginia_labels_v4', 'jobId': 'clr6tewjh000e08l45f5h9xmh'}
processing: sleeping
processing

processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2024-01-09 16:03:08.971868
tilesets upload-source stobieb virginia_al_sna_src_v4 virginia_al_sna_v4.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/virginia_al_sna_src_v4", "files": 1, "source_size": 58253581, "file_size": 58253581}', '']
tilesets create stobieb.virginia_al_sna_v4 --recipe virginia_al_sna_recipe_v4.json --name virginia_al_sna_v4
['{"message": "stobieb.virginia_al_sna_v4 already exists"}', ''] 
tilesets update-recipe stobieb.virginia_al_sna_v4 virginia_al_sna_recipe_v4.json
[''] Updated recipe.

tilesets publish stobieb.virginia_al_sna_v4
2024-01-09 16:03:41.808177
['{"message": "Processing stobieb.virginia_al_sna_v4", "jobId": "clr6ua2is003908l8g616c11v"}', ''] 
✔ Tileset job received. Visit https://studio

[''] Updated recipe.

tilesets publish stobieb.virginia_as_pna_v4
2024-01-09 16:24:00.569907
['{"message": "Processing stobieb.virginia_as_pna_v4", "jobId": "clr6v06vt000a08jte6d4h5gb"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.virginia_as_pna_v4 or run tilesets job stobieb.virginia_as_pna_v4 clr6v06vt000a08jte6d4h5gb to view the status of your tileset.

{'message': 'Processing stobieb.virginia_as_pna_v4', 'jobId': 'clr6v06vt000a08jte6d4h5gb'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2024-01-09 16:26:33.479038
tilesets upload-source stobieb virginia_as_sna_src_v4 virginia_as_sna_v4.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/virginia_as_sna_src_v4", "files": 1, "source_size": 58229167, "file_size": 58229167}', '']
tilesets create stobieb.virginia_as_sna_v4 --reci

['upload progress', '{"id": "mapbox://tileset-source/stobieb/virginia_bl_pna_src_v4", "files": 1, "source_size": 57788133, "file_size": 57788133}', '']
tilesets create stobieb.virginia_bl_pna_v4 --recipe virginia_bl_pna_recipe_v4.json --name virginia_bl_pna_v4
['{"message": "stobieb.virginia_bl_pna_v4 already exists"}', ''] 
tilesets update-recipe stobieb.virginia_bl_pna_v4 virginia_bl_pna_recipe_v4.json
[''] Updated recipe.

tilesets publish stobieb.virginia_bl_pna_v4
2024-01-09 16:46:40.714517
['{"message": "Processing stobieb.virginia_bl_pna_v4", "jobId": "clr6vtccc005u08l80jtd941z"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.virginia_bl_pna_v4 or run tilesets job stobieb.virginia_bl_pna_v4 clr6vtccc005u08l80jtd941z to view the status of your tileset.

{'message': 'Processing stobieb.virginia_bl_pna_v4', 'jobId': 'clr6vtccc005u08l80jtd941z'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
p

processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2024-01-09 17:09:49.819774
tilesets upload-source stobieb virginia_hi_pna_src_v4 virginia_hi_pna_v4.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/virginia_hi_pna_src_v4", "files": 1, "source_size": 57352853, "file_size": 57352853}', '']
tilesets create stobieb.virginia_hi_pna_v4 --recipe virginia_hi_pna_recipe_v4.json --name virginia_hi_pna_v4
['{"message": "stobieb.virginia_hi_pna_v4 already exists"}', ''] 
tilesets update-recipe stobieb.virginia_hi_pna_v4 virginia_hi_pna_recipe_v4.json
[''] Updated recipe.

tilesets publish stobieb.virginia_hi_pna_v4
2024-01-09 17:10:24.287911
['{"message": "Processing stobieb.virginia_hi_pna_v4", "jobId": "clr6wnw0s000708l5dz7if46c"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/

['{"message": "Processing stobieb.virginia_ot_sch_v4", "jobId": "clr6xdckx002108l72za3fwox"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.virginia_ot_sch_v4 or run tilesets job stobieb.virginia_ot_sch_v4 clr6xdckx002108l72za3fwox to view the status of your tileset.

{'message': 'Processing stobieb.virginia_ot_sch_v4', 'jobId': 'clr6xdckx002108l72za3fwox'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2024-01-09 17:33:19.218757
tilesets upload-source stobieb virginia_ot_pna_src_v4 virginia_ot_pna_v4.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/virginia_ot_pna_src_v4", "files": 1, "source_size": 57346960, "file_size": 57346960}', '']
tilesets create stobieb.virginia_ot_pna_v4 --recipe virginia_ot_pna_recipe_v4.json --name virginia_o

['{"message": "stobieb.virginia_wh_sch_v4 already exists"}', ''] 
tilesets update-recipe stobieb.virginia_wh_sch_v4 virginia_wh_sch_recipe_v4.json
[''] Updated recipe.

tilesets publish stobieb.virginia_wh_sch_v4
2024-01-09 17:53:43.197919
['{"message": "Processing stobieb.virginia_wh_sch_v4", "jobId": "clr6y7k1v000208k30v2n6tgd"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.virginia_wh_sch_v4 or run tilesets job stobieb.virginia_wh_sch_v4 clr6y7k1v000208k30v2n6tgd to view the status of your tileset.

{'message': 'Processing stobieb.virginia_wh_sch_v4', 'jobId': 'clr6y7k1v000208k30v2n6tgd'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2024-01-09 17:56:34.311803
tilesets upload-source stobieb virginia_wh_pna_src_v4 virginia_wh_pna_v4.json --replace
['upload progress', '{"id": "mapb

In [16]:
import csv
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie"
published_data_base_path = f'/Users/alanmccann/Dropbox/bain/mtsds-data/v{MAP_VERSION}'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings_v{MAP_VERSION}.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)

In [61]:
for row in market_rows:
    print(row["MarketName"],len(row["MarketName"]))

AbileneTX 9
AkronOH 7
AlbanyGA 8
AlbanyOR 8
AlbanyNY 8
AlbuquerqueNM 13
AlexandriaLA 12
AllentownPA 11
AltoonaPA 9
AmarilloTX 10
AmesIA 6
AnchorageAK 11
AnnArborMI 10
AnnistonAL 10
AppletonWI 10
AshevilleNC 11
AthensGA 8
AtlantaGA 9
AtlanticCityNJ 14
AuburnAL 8
AugustaGA 9
AustinTX 8
BakersfieldCA 13
BaltimoreMD 11
BangorME 8
BarnstableTownMA 16
BatonRougeLA 12
BattleCreekMI 13
BayCityMI 9
BeaumontTX 10
BeckleyWV 9
BellinghamWA 12
BendOR 6
BillingsMT 10
BinghamtonNY 12
BirminghamAL 12
BismarckND 10
BlacksburgVA 12
BloomingtonIL 13
BloomingtonIN 13
BloomsburgPA 12
BoiseCityID 11
BostonMA 8
BoulderCO 9
BowlingGreenKY 14
BremertonWA 11
BridgeportCT 12
BrownsvilleTX 13
BrunswickGA 11
BuffaloNY 9
BurlingtonNC 12
BurlingtonVT 12
CaliforniaMD 12
CantonOH 8
CapeCoralFL 11
CapeGirardeauMO 15
CarbondaleIL 12
CarsonCityNV 12
CasperWY 8
CedarRapidsIA 13
ChambersburgPA 14
ChampaignIL 11
CharlestonWV 12
CharlestonSC 12
CharlotteNC 11
CharlottesvilleVA 17
ChattanoogaTN 13
CheyenneWY 10
ChicagoIL 9
Ch

In [46]:
market_rows[155]['MarketName']

'HomosassaSprngFL'

In [122]:
STATS_TYPES[:2]

['Count', 'Percent']

In [133]:
for ethnicity in ETHNICITIES.keys():
    print(ethnicity)

all
asian
black
hispanic
other
white


In [136]:
for stats_type in STATS_TYPES[:2]:
    print(stats_type)

Count
Percent


In [165]:
ETHNICITIES.keys()

dict_keys(['all', 'asian', 'black', 'hispanic', 'other', 'white'])

In [170]:
market_rows

[{'MarketName': 'AbileneTX',
  'Market_Display_Name': 'Abilene, TX',
  'MapboxMarketName': 'AbileneTX',
  'StateFips': '',
  'CenterLat': '32.453549',
  'CenterLong': '-99.730981',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8',
  'InterMarketMetro': '0'},
 {'MarketName': 'AbileneTX',
  'Market_Display_Name': 'Abilene, TX',
  'MapboxMarketName': 'AbileneTX',
  'StateFips': '',
  'CenterLat': '32.453549',
  'CenterLong': '-99.730981',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8',
  'InterMarketMetro': '0'},
 {'MarketName': 'AbileneTX',
  'Market_Display_Name': 'Abilene, TX',
  'MapboxMarketName': 'AbileneTX',
  'StateFips': '',
  'CenterLat': '32.453549',
  'CenterLong': '-99.730981',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8',
  'InterMarketMetro': '0'},
 {'MarketName': 'AbileneTX',
  'Market_Display_Name': 'Abilene, TX',
  'MapboxMarketName': 'AbileneTX',
  'StateFips': '',
  'CenterLat': '32.453549

In [167]:
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)

In [185]:
market_rows

[{'MarketName': 'AbileneTX',
  'Market_Display_Name': 'Abilene, TX',
  'MapboxMarketName': 'AbileneTX',
  'StateFips': '',
  'CenterLat': '32.453549',
  'CenterLong': '-99.730981',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8',
  'InterMarketMetro': '0'},
 {'MarketName': 'AshevilleNC',
  'Market_Display_Name': 'Asheville, NC',
  'MapboxMarketName': 'AshevilleNC',
  'StateFips': '',
  'CenterLat': '35.595548',
  'CenterLong': '-82.548192',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8',
  'InterMarketMetro': '1'},
 {'MarketName': 'CharlestonSC',
  'Market_Display_Name': 'Charleston, SC',
  'MapboxMarketName': 'CharlestonSC',
  'StateFips': '',
  'CenterLat': '32.77657',
  'CenterLong': '-79.931085',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8',
  'InterMarketMetro': '1'},
 {'MarketName': 'DallasFortWorthTX',
  'Market_Display_Name': 'Dallas Fort Worth, TX',
  'MapboxMarketName': 'DallasFortWorthTX',
  'S

In [21]:
STATS_TYPES

['Count',
 'Percent',
 'nshft',
 'pshft',
 'growth',
 'pshftna',
 'growthna',
 'Labels']

In [4]:
stats_types = ['Count','Percent','CountChange', 'PercentChange','SaturationChange', 'PercentChangeNatAdj','SaturationChangeNatAdj','Labels']

In [5]:
stats_types[:-1]

['Count',
 'Percent',
 'CountChange',
 'PercentChange',
 'SaturationChange',
 'PercentChangeNatAdj',
 'SaturationChangeNatAdj']